# Needleman-Wunsch Algorithm

3차원 배열로 시뮬레이션을 구현하여 서열 정렬 과정의 정보와 가능한 모든 경우를 담은 배열을 생성하자!

In [51]:
SEQUENCE_1 = input('정렬할 염기서열 입력: ')
SEQUENCE_2 = input('기준 염기서열 입력: ')
# SEQUENCE_1 = 'GCCAAGACAGTATCAGACAC' # 정렬할 염기서열
# SEQUENCE_2 = 'TGTCATGCCAAGACAGTATG' # 기준 염기서열

MATRIX_ROW_N = len(SEQUENCE_1)+1 # 행 크기 초기화 
MATRIX_COLUMN_N = len(SEQUENCE_2)+1 # 열 크기 초기화
MATCH_SCORE = 8 # Match Score
MISMATCH_SCORE = -5 # Mismatch Score
GAP_SCORE = -3 # Gap Points
GAP_CHAR = '-' # 공백 표시
ALN_PATHWAYS = [] # 찾은 정렬 방법들 저장하는 리스트 초기화
MATRIX = [[[[0] for i in range(2)] for i in range(MATRIX_COLUMN_N)] for i in range(MATRIX_ROW_N)]

'''
Score matrix로 3차원 배열을 구현. 첫 번째 차원: 행 인덱스, 두 번째 차원: 열 인덱스, 세 번째 차원: 리스트의 0번 인덱스에 현재 위치에서의 점수, 1번 인덱스에 진행 가능한 방향의 목록을 담음.
예시로 MATRIX[2][3]은 행 인덱스가 2, 열 인덱스가 3인 위치에서의 정보이다. MATRIX[2][3][0]에는 현재 위치에서의 점수가, MATRIX[2][3][1]에는 현재 위치에서 진행 가능한 방향의 목록이 있다.
'''

def print_aln_details(aln): 
    print('\n>>>>>>>>>>>>>>>> Aln #:'+str(aln[4])+' <<<<<<<<<<<<<<<<\n\n------------------Steps------------------\n')
    for elem in aln[3]:
        print('Step: '+str(elem[0])+' -> Score = ' + str(elem[1])    + ' -> Dir: ' + ('\u2190' if elem[2]==1 else ('\u2196' if elem[2]==2 else ('\u2191' if elem[2]==3 else 'Error'))))
    print('\n-------------Final aln-------------\n')
    print(aln[1]+'\n'+aln[0])
    return


def print_all(ALIGNMENTS):  
    print('Total Alignments: ' + str(len(ALIGNMENTS)))
    print('Overall Score: '+str(ALIGNMENTS[0][3][0][1])+'\n')
    for elem in ALIGNMENTS:
        print_aln_details(elem)
    return

def print_alns_only(ALIGNMENTS): 
    print('Total Alignments: ' + str(len(ALIGNMENTS)))
    print('Overall Score: '+str(ALIGNMENTS[0][3][0][1])+'\n')
    for elem in ALIGNMENTS:
        print(elem[0]+'\n'+elem[1]+'\n')
    return

# ------------------ 백트래킹 함수 ------------------
def find_each_path(c_i,c_j,path=''): # 가능한 모든 정렬 경로 찾기
    global ALN_PATHWAYS
    i, j = c_i, c_j # 현재 위치 (c_i, c_j)에서 시작
    
    if i == 0 and j==0: # 함수가 (0, 0) 위치까지 도달하여 경로 추가 후 종료
        ALN_PATHWAYS.append(path) # path는 현재까지의 경로를 나타내는 문자열
        return 2 # 경로가 끝났을 때의 함수 종료 신호
    
    dir_t = len(MATRIX[i][j][1]) # 현재 위치에서 진행 가능한 방향의 개수 
    
    while dir_t <= 1: # 진행 가능한 방향이 하나 이하일 떄까지
        n_dir = MATRIX[i][j][1][0] if (i != 0 and j != 0) else (1 if i == 0 else (3 if j==0 else 0)) # 진행 방향 결정. 만약 (i, j)가 행렬 내부에 있는 위치라면 MATRIX[i][j][1][0]에서 첫 번째로 기록된 방향 선택. (i, j)가 가장자리면 행 
        path = path + str(n_dir) 
        if n_dir == 1: # 왼쪽 방향
            j = j-1
        elif n_dir == 2: # 왼쪽 위 방향
            i = i-1
            j = j-1
        elif n_dir == 3: # 위쪽 방향
            i = i-1
        dir_t = len(MATRIX[i][j][1]) # 새로운 위치에서의 진행 가능 방향 개수 업데이트
        if i == 0 and j==0: # 배열의 시작 위치에 도달하면
            ALN_PATHWAYS.append(path) # 현재까지의 경로를 추가
            return 3 # (0, 0) 이 아닌 다른 위치에서의 경로(추가적 탐색 필요 신호)
        
    if dir_t > 1: # 여러 개의 방향 중 하나 선택해서 이동
        for dir_c in range(dir_t): # 가능한 방향 모두에 대해서 반복
            n_dir = MATRIX[i][j][1][dir_c] if (i != 0 and j != 0) else (1 if i == 0 else (3 if j==0 else 0))
            tmp_path = path + str(n_dir) # 현재까지의 경로에 선택한 방향을 추가한 임시 경로
            if n_dir == 1: 
                n_i = i
                n_j = j-1
            elif n_dir == 2:
                n_i = i-1
                n_j = j-1
            elif n_dir == 3:
                n_i = i-1
                n_j = j
            find_each_path(n_i,n_j,tmp_path) # 재귀 호출
            
    return len(ALN_PATHWAYS)


#Main Code
# 배열 초기화
for i in range(MATRIX_ROW_N):
    MATRIX[i][0] = [GAP_SCORE*i,[]]
for j in range(MATRIX_COLUMN_N):
    MATRIX[0][j] = [GAP_SCORE*j,[]]
for i in range(1,MATRIX_ROW_N):
    for j in range(1,MATRIX_COLUMN_N):
        score = MATCH_SCORE if (SEQUENCE_1[i-1] == SEQUENCE_2[j-1]) else MISMATCH_SCORE # 일치하면 +8, 다르면 -5
        h_val = MATRIX[i][j-1][0] + GAP_SCORE # 가로로 이동
        d_val = MATRIX[i-1][j-1][0] + score # 대각선으로 이동
        v_val = MATRIX[i-1][j][0] + GAP_SCORE # 세로로 이동
        o_val = [h_val, d_val, v_val]
        MATRIX[i][j] = [max(o_val), [i+1 for i,v in enumerate(o_val) if v==max(o_val)]] # 이들 중 최댓값 점수들과 이동 가능한 방향의 리스트 저장. 예시로 o_val = [10, 5, 10] 이면 MATRIX[i][j] = [10, [1, 3]]

score = MATRIX[i][j][0] # 마지막 위치에서의 점수 = 76

l_i = i
l_j = j
ALIGNMENTS = []
tot_aln = find_each_path(i,j) # 함수 실행
aln_count = 0

for elem in ALN_PATHWAYS: # ['33332332222222222121112121', '33332332222222222211112121', '33332332222222222221111121', '33332332222222222222111111'] <- elems
    i = l_i-1
    j = l_j-1
    side_aln = '' # 정렬된 서열
    top_aln = '' # 기준 서열
    step = 0
    aln_info = []
    for n_dir_c in range(len(elem)):
        n_dir = elem[n_dir_c]
        score = MATRIX[i+1][j+1][0]
        step = step + 1
        aln_info.append([step,score,n_dir]) # [[1, 76, '3'], [2, 76, '3'], [3, 76, '3'], [4, 76, '3'], [5, 76, '2']...
        if n_dir == '2':
            side_aln = side_aln + SEQUENCE_1[i]
            top_aln = top_aln + SEQUENCE_2[j]
            i=i-1
            j=j-1
        elif n_dir == '1':
            side_aln = side_aln + GAP_CHARACTER
            top_aln = top_aln + SEQUENCE_2[j]
            j=j-1
        elif n_dir == '3':
            side_aln = side_aln + SEQUENCE_1[i]
            top_aln = top_aln + GAP_CHARACTER
            i=i-1
    aln_count = aln_count + 1 # 정렬 결과 개수 증가
    ALIGNMENTS.append([top_aln[::-1],side_aln[::-1],elem,aln_info,aln_count]) # 서열 역순 바꿔 저장

print_alns_only(ALIGNMENTS)

Total Alignments: 4
Overall Score: 76

TGTCATGCCAAGACAGTAT--G----
-G-C---C-AAGACAGTATCAGACAC

TGTCATGCCAAGACAGTAT--G----
-G-C----CAAGACAGTATCAGACAC

TGTCATGCCAAGACAGTAT--G----
-G-----CCAAGACAGTATCAGACAC

TGTCATGCCAAGACAGTAT--G----
------GCCAAGACAGTATCAGACAC


# 추가 알고리즘 구현(Burrows-Wheeler Transform)

- BWA는 Burrosws-Wheeler Transform (BWT) 알고리즘을 적용하여 짧은 서열(200bp 미만)을 reference sequence에 정렬시키는 알고리즘이다. 이 과정에 적용된 BWT 알고리즘을 간단히 구현해보았다.
- BWT에서는 블록 정렬을 수행하며, 부분적으로 포함된 문자열의 패턴을 찾기 위해 사용된다. 

## BWT 과정 
1) 변환하고자 하는 텍스트의 맨 마지막에 단어의 끝을 알리는 토큰($)을 넣는다.
2) 토큰을 넣은 문자열을 왼쪽으로 한 칸씩 이동하여 배열을 생성한다.(순환 회전)
3) 생성된 배열을 첫 번 째 열을 기준으로 사전 순 정렬을 한다.
4) 정렬된 행렬의 맨 마지막 열의 문자들로 생성된 문자열이 변환된 결과이다. 

In [64]:
matrixRot = []  
F = ''  # 각 회전 변환의 첫 번째 문자를 포함하는 마지막 열 
L = ''  # 각 회전 변환의 마지막 문자를 포함하는 첫 번째 열
lastCol = []  # 문자와 위치를 담은 튜플 목록
firstCol = [] 
array = tuple()   

def ranker(volatile,i):
    if i == '$': # 마지막 문자가 토큰일 경우
        output = '$'   
        return output
    else:  # 토큰이 아닐 경우
        output = volatile.count(i) - 1  # 선택한 slice에 포함된 개수 세기 -1 (인덱스는 0부터 시작)
        return output

def suffix(M): # 입력 문자열의 각 회전에서 첫 번째 문자로부터 토큰 "$"까지 각 부분 문자열의 오프셋을 계산.
    global array
    for t in M: # 배열 안의 하나 회전된 배열 선택 
        i = 1
        suffix = len(t) # 각각 회전시킬때마다 길이 저장
        while True:
            if t[i-1] == '$':  # 만약 토큰이라면
                suffix = suffix - i   # 현재 위치를 길이에서 뺴기
                array += (suffix,)  # 계산 결과를 배열에 추가 
                break  # 토큰 찾으면 실행 중단 
            else:
                i += 1  # 토큰 찾을 때까지 반복
    return array  

def cyclicRotations(T): # 순환 회전 변환
    global matrixRot
    TT = T * 2
    matrixRot = [TT[i:i+len(T)] for i in range(0,len(T))]  # 순환 회전된 배열 생성
    matrixRot = sorted(matrixRot)  # 사전순 정렬
    array = suffix(matrixRot) 
    return matrixRot  # Burrows-Wheeler Matrix(BWM)

def BWT(T): 
    global F
    global L
    T = T + '$' # 끝에 토큰 추가
    rotations = cyclicRotations(T) 
    for e in rotations: # 각 순환된 배열에서 첫 번째 문자, 마지막 문자를 가져와서 
        L += e[-1] # 마지막 문자 추가
        F += e[0] # 첫 번째 문자 추가
    return L

def firstColRank(F,LL): # BWM의 첫 번째 문자들에 순위 지정
    global firstCol  
    for y in F: 
        j = 0
        while True:
            currentCell = tuple() # 문자와 순서 담는 튜플
            if LL[j][0] == y: # 마지막 열에서 순위를 매긴 첫 번째 요소와 순위를 매기지 않은 첫 번째 열의 첫 번째 요소를 선택
                currentCell = (y,) + (LL[j][1],) # 마지막 열의 첫 번째 문자와 첫 번째 열의 첫 번째 문자에 동일한 순위를 연결
                firstCol += (currentCell,) # 새로 문자-순위를 연결한 첫 번째 열
                LL.remove(LL[j])  # 첫 번째 열의 문자가 중복되는 오류 막기 위해 선택한 마지막 열의 문자를 제거
                break  
            else:
                j += 1 # 다음 문자 시행 
    return firstCol

def T_ranking():  # BWM의 첫 번쨰 열과 마지막 열의 순위 구축
    global lastCol
    ranking_L = []  # 마지막 열의 순위
    i = 0
    for v in matrixRot:
        slicer = v.find('$')  
        volatile = v[slicer:]  # 특수문자에서 끝까지의 부분문자열 선택
        currentRank = ranker(volatile, L[i]) # 각 순환 회전된 배열에서 마지막 문자 순위 매기기
        ranking_L += [currentRank,] # 현재 순위 추가 
        i += 1
    i = 0
    while i != len(L):
        currentCell = (L[i],) + (ranking_L[i],)  # 문자-순위 연결한 튜플 추가
        lastCol += (currentCell,)  # 마지막 열에 추가
        i += 1
    LL = lastCol[:]
    firstCol = firstColRank(F,LL)  # 마지막 열(문자-순위)을 참조해서 첫 번째 열의 문자 순위를 생성
    return firstCol,lastCol     

def reverseBWT(T):  # BWT 역순으로 정렬하여 원본 문자열 얻기
    columns = T_ranking()  # 함수 호출 
    reverse = ''
    guard = columns[1][0] # 마지막 열의 첫 째 문자에서 시작
    while guard != ('$','$'):  # 토큰 나올 때까지
        reverse += guard[0] # 현재의 마지막 문자 추가 
        index = columns[0].index(guard) # 현재 열 기준으로 마지막 열과 동일한 문자 있는 열의 인덱스 얻기
        guard = columns[1][index] # 이전에 찾은 인덱스에 해당하는 마지막 열의 문자로 이동

    return reverse[::-1] # 역순 정렬

def stringMatch(P):  # 문자열을 받아 해당 문자열이 나타나는 위치와 일치 범위 반환
    global firstCol
    global lastCol
    global array

    i = len(P)-1  # 일치시킬 문자열의 마지막 문자부터 시작
    start = []
    suffix = P[i]  
    for x in range(len(firstCol)): 
        vol = tuple()
        if firstCol[x][0] == suffix: # 마지막 열의 문자-순위와 일치한다면
            vol = (firstCol[x],) + (x,) # 일치하는 시작 지점 중 하나로 저장한다.
            start += (vol,)

    if len(P) == 1:  # 부분문자열 길이가 1이라면
        indexForRange = [] # 순위가 매겨진 첫 번째 열에서 초기 
        count = 0
        if len(start) == 0: 
            return count,indexForRange
        else:
            for char in start:  # 문자열에 적어도 한 번 나타나면
                count += 1 # 발생 횟수 세고
                index = firstCol.index(char[0]) # BWM에서 발생 위치 얻기 
                indexForRange += [index,] # 이를 이용해 발생 범위 추정

    else: # 부분문자열 길이가 2 이상이라면
        count = 0
        indexForRange = []  
        for e in start: 
            i = len(P)-1    
            index = e[1]  # 첫 번째 현재 열 순위의 값 설정
            while True:
                i = i - 1  # 부분문자열의 다음 문자열로 이동
                guard = lastCol[index]  # 첫 번째 열의 문자-순위에 해당하는 위치에 있는 마지막 열의 문자 선택
                if guard[0] == P[i]: # 만약 열-순위 튜플의 문자가 현재 문자와 같다면
                    index = firstCol.index(guard) # 새 문자로 이동
                    if i == 0: # prefix가 최대 길이에 도달해 일치 발견
                        indexForRange += [index,] # 부분 문자열의 마지막 문자가 일치한 BWM의 인덱스 선택
                        count += 1 # 일치 수 증가
                        break
                else:
                    break  # 일치하지 않으면 다음 시작 지점으로 

    rangeSub = []  # 각 부분 문자열의 일치에 대한 시작-끝 인덱스를 채울 리스트
    for r in indexForRange:
        rank = []  # 현재 일치의 범위
        loc = array[r] 
        rank = [loc,(loc+len(P)-1)] # 발생의 시작과 끝 선택
        rangeSub += [rank,]  # 현재 범위 추가

    return count,sorted(rangeSub) # 발생 횟수와 각각의 문자열에서의 범위를 보고

T = input('BWT 변환할 문자열 입력:')
T = T.upper()
BWT = BWT(T)
print('BWT 변환 결과:',BWT)
reverse = reverseBWT(T)
print('역순으로 정렬한 BWT 변환 결과:', reverse)

subString = input('찾으려는 부분 문자열 입력:')
subString = subString.upper() 
print('찾으려는 부분 문자열:', subString)
match = stringMatch(subString)
if match[0] != 0:
    print(f'{match[1]} 인덱스에서 {match[0]}번 부분 일치 발생')
else:
    print('일치하는 부분이 없다')


BWT 변환 결과: SHGY$APBP
역순으로 정렬한 BWT 변환 결과: HAPPYGBS
찾으려는 부분 문자열: GBS
[[5, 7]] 인덱스에서 1번 부분 일치 발생
